Analyse du portefeuille de brevets de CNIM 662043595



1) - dynamique de l'innovation
-> nombre de brevets par an, largeur de la famille au cours du temps

Je m'intéresse à sa quantité d'innovation, et non sa qualité, je ne prends donc pas en compte le fait qu'un brevet soit délivré ou non. 

In [ ]:
--liste de tous les brevets de CNIM GROUPE
Select distinct a.docdb_family_id as brevet ,a.granted,appln_filing_year as year, appln_auth as country,han_name 
from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
where (han_name like 'CNIM GROUPE%' or han_name like 'CONSTRUCTIONS IND DE LA MÉDITERRANÉE%')
and psn_sector = 'COMPANY'


In [ ]:
-- nb demandes par famille de demande
Select a.docdb_family_id, count(distinct a.appln_id) as nb_demande
from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
where han_name like 'CNIM GROUPE%' 
and psn_sector = 'COMPANY'
group by  a.docdb_family_id

In [ ]:
--nombre de brevets par an
Select appln_filing_year as year, count(distinct a.docdb_family_id) as nb_brevets 
from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
where (han_name like 'CNIM GROUPE%' or han_name like 'CONSTRUCTIONS IND DE LA MÉDITERRANÉE%')
and psn_sector = 'COMPANY'
group by appln_filing_year
order by year

2) - géographie: pour un même brevet (donc d'une meme famille et du meme auteur) 

In [ ]:
select appln_auth as country, appln_filing_year as year,han_name,
count(distinct a.appln_id) as number_demande
 from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
where han_name like 'CNIM GROUPE%' 
and psn_sector = 'COMPANY'
group by appln_auth,appln_filing_year,han_name
order by appln_filing_year

3) - diversité des champs d'application

Pour se faire j'utilise l'IPC class par application

In [ ]:
Select left(ipc.ipc_class_symbol,4) as IPC, count(distinct a.appln_id) as number_demandes 
from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
left join tls209_appln_ipc ipc on a.appln_id = ipc.appln_id
where (han_name like 'CNIM GROUPE%' or han_name like 'CONSTRUCTIONS IND DE LA MÉDITERRANÉE%')
and psn_sector = 'COMPANY'
group by left(ipc.ipc_class_symbol,4)
order by count(distinct a.appln_id) desc 

nombre d'ipc par application

In [ ]:
Select  a.appln_id, count(left(ipc.ipc_class_symbol,4))as nb_IPC
from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
left join tls209_appln_ipc ipc on a.appln_id = ipc.appln_id
where han_name like 'CNIM GROUPE%'
and psn_sector = 'COMPANY'
group by a.appln_id


Pour obtenir la variance et la moyenne de chaque distribution de nombre de classe par année 

In [ ]:
SELECT 
    (AVG(nb_IPC_2) - POWER(AVG(nb_IPC), 2)) * (COUNT(*) / (COUNT(*) - 1)) AS variance,
    AVG(nb_IPC) AS moyenne 
FROM (
    SELECT  
        a.appln_id, 
        COUNT(LEFT(ipc.ipc_class_symbol, 4)) AS nb_IPC,  
        POWER(COUNT(LEFT(ipc.ipc_class_symbol, 4)), 2) AS nb_IPC_2
    FROM 
        tls201_appln a
    LEFT JOIN 
        tls209_appln_ipc ipc 
    ON 
        a.appln_id = ipc.appln_id
    WHERE 
        a.appln_filing_year = '2019'
    GROUP BY 
        a.appln_id
) subquery;


Nombre de fois que chaque classe IPC a été utilisée annuellement

In [ ]:
Select a.appln_filing_year, left(ipc.ipc_class_symbol,4) as IPC, count(left(ipc.ipc_class_symbol,4) ) as nb_IPC
from tls201_appln a
inner join tls207_pers_appln p on a.appln_id = p.appln_id
inner join tls206_person i on  p.person_id = i.person_id
left join tls209_appln_ipc ipc on a.appln_id = ipc.appln_id
where han_name like 'CNIM GROUPE%'
and psn_sector = 'COMPANY'
group by a.appln_filing_year, left(ipc.ipc_class_symbol,4)
order by a.appln_filing_year,left(ipc.ipc_class_symbol,4)

4. Positionnement stratégique: radical ou incrémental

Forward et backward citations

S'il n'y a pas beaucoup de backward et beaucoup de forward-> invention radicale

Les backward citations

In [ ]:
--liste des backwards 
SELECT 
    DISTINCT 
    a.docdb_family_id,
    a.appln_id,
    a.appln_filing_year AS anneeA,      -- Année de dépôt
   count( cit.cited_pat_publn_id) AS nb_backwards -- backward par famille de brevets
 
FROM 
    tls201_appln a
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
inner join tls211_pat_publn t on t.appln_id = a.appln_id
inner JOIN 
 tls212_citation cit ON t.pat_publn_id = cit.pat_publn_id --backwards
WHERE 
    i.han_name LIKE 'CNIM GROUPE%' -- Filtrer sur le groupe
    AND a.ipr_type = 'PI'         -- Type de propriété intellectuelle
    AND p.applt_seq_nr > 0        -- Vérifier que c'est le déposant principal
    AND p.invt_seq_nr = 0         -- Exclure les inventeurs
   
   group by   a.docdb_family_id, a.appln_id, a.appln_filing_year
    
ORDER BY 
    a.docdb_family_id,
    a.appln_id,
    a.appln_filing_year
    
    



Obtenir la moyenne et la variance du nombre de backwards de la cohorte de chaque demande de brevet

In [ ]:
SELECT 
    (AVG(nb_backwards_2) - POWER(AVG(nb_backwards), 2)) * (1.0 * COUNT(*) / (COUNT(*) - 1)) AS variance,
    AVG(nb_backwards) AS moyenne 
FROM (
    SELECT  
        a.appln_id, 
        count( cit.cited_pat_publn_id) AS nb_backwards,
        POWER( 1.0* count(cit.cited_pat_publn_id), 2) AS nb_backwards_2
    FROM 
        tls201_appln a
 inner join tls211_pat_publn t on t.appln_id = a.appln_id
inner JOIN 
 tls212_citation cit ON t.pat_publn_id = cit.pat_publn_id 
    WHERE 
        a.appln_filing_year = '2020'
    GROUP BY 
        a.appln_id
) subquery;


Nombre de forwards

In [ ]:
Select a.docdb_family_id, count(distinct f.docdb_family_id) as nb_forward
from tls201_appln a 
inner join tls228_docdb_fam_citn f on cited_docdb_family_id = a.docdb_family_id
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
WHERE 
    i.han_name LIKE 'CNIM GROUPE%' -- Filtrer sur le groupe
    AND a.ipr_type = 'PI'         -- Type de propriété intellectuelle
    AND p.applt_seq_nr > 0        -- Vérifier que c'est le déposant principal
    AND p.invt_seq_nr = 0    
group by a.docdb_family_id


6° Les concurrents et les positionner 

In [ ]:
--liste des backwards 
Select i.han_name, left(ipc.ipc_class_symbol,4) as ipc, count(distinct a.appln_id) as nb_backwards
FROM 
    tls201_appln a
inner join tls211_pat_publn pat on 	a.appln_id = pat.appln_id
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
left join tls209_appln_ipc ipc on a.appln_id = ipc.appln_id
where pat.pat_publn_id in 
(SELECT 
 	distinct cit.cited_pat_publn_id AS backwards -- backward 
FROM 
    tls201_appln a
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
inner join tls211_pat_publn t on t.appln_id = a.appln_id
inner JOIN 
 tls212_citation cit ON t.pat_publn_id = cit.pat_publn_id --backwards
WHERE 
    i.han_name LIKE 'CNIM GROUPE%' -- Filtrer sur le groupe
    AND a.ipr_type = 'PI'         -- Type de propriété intellectuelle
    AND p.applt_seq_nr > 0        -- Vérifier que c'est le déposant principal
    AND p.invt_seq_nr = 0         -- Exclure les inventeurs
   )
   
   group by i.han_name, left(ipc.ipc_class_symbol,4)
   order by nb_backwards desc

    

    



7° qualité des brevets

grant lag:

In [ ]:
SELECT 
    DISTINCT a.appln_id, 
    DATEDIFF(day, a.earliest_filing_date,t.publn_date) AS filing_to_publication_days
FROM 
    tls201_appln a
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
INNER JOIN 
    tls211_pat_publn t ON a.earliest_pat_publn_id = t.pat_publn_id
WHERE 
    i.han_name LIKE 'CNIM GROUPE%'
    AND a.ipr_type = 'PI'       
    AND p.applt_seq_nr > 0      
    AND p.invt_seq_nr = 0        
    AND a.granted = 'Y'


grant lag cohortes:

In [ ]:
SELECT  
    (AVG(CAST(lag_grant_2 AS BIGINT)) - POWER(AVG(CAST(lag_grant AS FLOAT)), 2)) * ((1.0 * COUNT(*)) / (COUNT(*) - 1)) AS variance,
    AVG(CAST(lag_grant AS FLOAT)) AS moyenne 
FROM (
    SELECT 
        a.appln_id, 
        DATEDIFF(day, a.earliest_filing_date, t.publn_date) AS lag_grant,
        POWER(1.0 * CAST(DATEDIFF(day, a.earliest_filing_date, t.publn_date) AS BIGINT), 2) AS lag_grant_2
    FROM 
        tls201_appln a
    INNER JOIN 
        tls211_pat_publn t ON a.earliest_pat_publn_id = t.pat_publn_id
    WHERE 
        a.appln_filing_year = '2015'
        and granted = 'Y'
) subquery;


nb de revendications

In [ ]:
SELECT 
  distinct a.appln_id, a.appln_filing_year,t.publn_claims
FROM 
    tls201_appln a
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
INNER JOIN 
    tls211_pat_publn t ON a.appln_id = t.appln_id
WHERE 
    i.han_name LIKE 'CNIM GROUPE%'
    AND a.ipr_type = 'PI'       
    AND p.applt_seq_nr > 0      
    AND p.invt_seq_nr = 0        


comparer avec les cohortes

In [ ]:
SELECT  
    (AVG(CAST(claims_2 AS BIGINT)) - POWER(AVG(CAST(claims AS FLOAT)), 2)) * ((1.0 * COUNT(*)) / (COUNT(*) - 1)) AS variance,
    AVG(CAST(claims AS FLOAT)) AS moyenne 
FROM (
    SELECT 
        a.appln_id, 
        t.publn_claims AS claims, 
        POWER(1.0 * t.publn_claims, 2) AS claims_2
    FROM 
        tls201_appln a
    INNER JOIN 
        tls207_pers_appln p ON a.appln_id = p.appln_id
    INNER JOIN 
        tls206_person i ON p.person_id = i.person_id
    INNER JOIN 
        tls211_pat_publn t ON a.appln_id = t.appln_id
    WHERE 
        a.appln_filing_year = '2019'
        AND t.publn_claims != 0
) subquery;


Indice d'originalité

In [ ]:
SELECT 
 	 a.appln_id,left(ipc.ipc_class_symbol,4) as ipc, count(*) --ipc des backwards 
FROM 
    tls201_appln a
INNER JOIN 
    tls207_pers_appln p ON a.appln_id = p.appln_id
INNER JOIN 
    tls206_person i ON p.person_id = i.person_id
inner join tls211_pat_publn t on t.appln_id = a.appln_id
inner JOIN 
 tls212_citation cit ON t.pat_publn_id = cit.pat_publn_id --backwards
 inner join tls211_pat_publn cit2 on cit.cited_pat_publn_id = cit2.pat_publn_id
 inner join tls201_appln cit3 on cit2.appln_id = cit3.appln_id
 inner join tls209_appln_ipc ipc on ipc.appln_id = cit3.appln_id
WHERE 
    i.han_name LIKE 'CNIM GROUPE%' -- Filtrer sur le groupe
    AND a.ipr_type = 'PI'         -- Type de propriété intellectuelle
    AND p.applt_seq_nr > 0        -- Vérifier que c'est le déposant principal
    AND p.invt_seq_nr = 0   
    group by  a.appln_id,left(ipc.ipc_class_symbol,4)